# 👟 Behaviors

Behaviors extend the appearance and interactivity of the [2D](#2D-Graph) and
[3D](./3D.ipynb) graphs. These can affect [nodes](#Node-Behaviors),
[links](#Link-Behaviors) or certain aspects of the graph itself.

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import random

import ipywidgets as W
import traitlets as T

import ipyforcegraph.behaviors as B

In [ ]:
with __import__("importnb").Notebook():
    from _index import make_a_simple_example

## 2D Graph

```{hint}
After rendering the cell below, select _Create New View For Output_ from the output's right-click menu to see more things added
```

In [ ]:
if __name__ == "__main__":
    fg, box = make_a_simple_example()
    display(box)

## Node Behaviors

### `NodeSelection`

The `NodeSelection` behavior allows for selecting one or more nodes from the browser, or
updating from the kernel.

In [ ]:
def add_node_selection(fg=None, box=None):
    if fg is None:
        fg, box = make_a_simple_example()
    selection = B.NodeSelection()
    fg.behaviors = [selection]
    ui_selection = W.TagsInput(
        placeholder="select some nodes",
        allowed_tags=sorted(fg.source.nodes.id),
    )
    T.link((selection, "selected"), (ui_selection, "value"))
    box.children[1].children += (ui_selection,)
    return fg, box

In [ ]:
if __name__ == "__main__":
    add_node_selection(fg, box)
    display(box)

```{hint}
Note that the nodes changed colors. Click a node to select it, or use <kbd>ctrl</kbd> or <kbd>shift</kbd> to select multiple nodes.
```

The selection is handed back from the client, and can be used with other widgets.

### `NodeColors`

Node colors can be set based on a column value. By default, a column named `color` will
be used. Ensure `NodeColors` is in `behaviors` _after_ e.g. `NodeSelection`.

In [ ]:
from ipyforcegraph.behaviors import NodeColors


def add_node_colors(fg=None, box=None, column_name="color"):
    if fg is None:
        fg, box = make_a_simple_example()
    colors = NodeColors(column_name=column_name)
    fg.behaviors = [*fg.behaviors, colors]
    if column_name not in fg.source.nodes:
        fg.source.nodes[column_name] = [
            "#" + "".join([random.choice("01234567abcdef") for j in range(6)])
            for i in range(len(fg.source.nodes))
        ]
        fg.source.send_state("nodes")
    ui_color_column = W.Dropdown(
        options=list(fg.source.nodes.columns), value=column_name
    )
    ui_color_template = W.Textarea()
    ui_color_template_enabled = W.Checkbox(description="enabled?")
    ui_colors = W.Accordion(
        [ui_color_column, W.VBox([ui_color_template_enabled, ui_color_template])],
        titles=["node color by column", "node color by template"],
    )
    T.link((ui_color_column, "value"), (colors, "column_name"))
    T.dlink(
        (ui_color_template, "value"),
        (colors, "template"),
        lambda x: x if ui_color_template_enabled.value else "",
    )
    box.children[1].children += (ui_colors,)
    return fg, box

In [ ]:
if __name__ == "__main__":
    add_node_colors(fg, box)
    display(box)

#### Color Templates

`NodeColors` support either a column name, and for the most part, calculating the values
derived for these as data frames is likely the best choice.

However, the`.template` traitlet, which take the form of
[nunjucks templates](https://mozilla.github.io/nunjucks/templating.html) allows for
calculating dynamic values on the client.

The syntax is intentionally very similar to
[jinja2](https://jinja.palletsprojects.com/en/3.1.x/templates).

Inside of a template, one can use:

- `node`
  - this will have all of the named columns available to it
- `graphData`
  - `nodes`
  - `links`
    - `source` and `target` as realized nodes

With these, and basic template tools, one can generate all kinds of interesting effects.
For the example data above, try these color templates:

- color by group
  > ```python
  > {{ ["red", "yellow", "blue", "orange", "purple", "magenta"][node.group] }}
  > ```
- color by out-degree
  > ```python
  > {% set n = 0 %}
  > {% for link in graphData.links %}
  >   {% if link.source.id == node.id %}{% set n = n + 1 %}{% endif %}
  > {% endfor %}
  > {% set c = 256 * (7-n) / 7 %}
  > rgb({{ c }},0,0)
  > ```

### `NodeLabels`

Node labels can be revealed when hovering over the node. By default the node's `id`
column will be used.

In [ ]:
from ipyforcegraph.behaviors import NodeLabels


def add_node_labels(fg=None, box=None, column_name="id"):
    if fg is None:
        fg, box = make_a_simple_example()
    labels = NodeLabels()
    ui_label_column = W.Dropdown(
        options=list(fg.source.nodes.columns), value=column_name
    )
    ui_label_template = W.Textarea()
    ui_label_template_enabled = W.Checkbox(description="enabled?")
    ui_labels = W.Accordion(
        [ui_label_column, W.VBox([ui_label_template_enabled, ui_label_template])],
        titles=["node label by column", "node label by template"],
    )
    T.link((ui_label_column, "value"), (labels, "column_name"))
    T.dlink(
        (ui_label_template, "value"),
        (labels, "template"),
        lambda x: x if ui_label_template_enabled.value else "",
    )
    box.children[1].children += (ui_labels,)
    fg.behaviors = [*fg.behaviors, labels]
    return fg, box

In [ ]:
if __name__ == "__main__":
    add_node_labels(fg, box)
    display(box)

#### Label templates

Like `NodeColors`, `NodeLabels` also accepts [templates](#color-templates). The
resulting value may be plain strings or HTML.

Here are some examples, again for the example data:

- just a header
  > ```html
  > <h1>{{ node.id }}</h1>
  > ```
- a table
  > ```html
  > <table>
  >  <tr><th>id</th><th>group</th></td>
  >  {% for link in graphData.links %}
  >  {% if link.source.id == node.id %}
  >  <tr><td>{{ link.target.id }}</td><td>{{ link.target.group }}</td>
  >  {% endif %}
  >  {% endfor %}
  > </table>
  > ```

## Link Behaviors

In [ ]:
def make_link_behavior_with_ui(WidgetClass, label, column_name, is_color=False):
    def add_behavior(fg, box, column_name=column_name):
        behavior = WidgetClass(column_name=column_name)
        fg.behaviors = [*fg.behaviors, behavior]
        if is_color:
            if column_name not in fg.source.links:
                fg.source.links[column_name] = [
                    "#" + "".join([random.choice("01234567abcdef") for j in range(6)])
                    for i in range(len(fg.source.links))
                ]
                fg.source.send_state("links")
        ui_column_name = W.Dropdown(
            options=list(fg.source.links.columns), value=column_name
        )
        ui_template = W.Textarea()
        ui_template_enabled = W.Checkbox(description="enabled?")
        ui = W.Accordion(
            [ui_column_name, W.VBox([ui_template_enabled, ui_template])],
            titles=[f"{label} by column", f"{label} by template"],
        )
        T.link((ui_column_name, "value"), (behavior, "column_name"))
        T.dlink(
            (ui_template, "value"),
            (behavior, "template"),
            lambda x: x if ui_template_enabled.value else "",
        )
        box.children[1].children += (ui,)
        return fg, box

    return add_behavior

### `LinkColors`

Link colors can also be configured by column name or template.

In [ ]:
add_link_colors = make_link_behavior_with_ui(B.LinkColors, "link color", "color", True)

In [ ]:
if __name__ == "__main__":
    add_link_colors(fg, box)
    display(box)

### `LinkLabels`

Link labels are revealed when hovered, and accept `column_name` or `template`.

In [ ]:
add_link_labels = make_link_behavior_with_ui(B.LinkLabels, "link label", "value")

if __name__ == "__main__":
    add_link_labels(fg, box)
    display(box)

### `LinkDirectionalArrowColor`

In [ ]:
add_link_directional_arrow_color = make_link_behavior_with_ui(
    B.LinkDirectionalArrowColor, "link directional arrow color", "color", is_color=True
)

if __name__ == "__main__":
    add_link_directional_arrow_color(fg, box)
    display(box)

### `LinkDirectionalArrowLength`

In [ ]:
add_link_directional_arrow_length = make_link_behavior_with_ui(
    B.LinkDirectionalArrowLength, "link directional arrow length", "value"
)

if __name__ == "__main__":
    add_link_directional_arrow_length(fg, box)
    display(box)

### `LinkDirectionalArrowRelPos`

In [ ]:
add_link_directional_arrow_rel_pos = make_link_behavior_with_ui(
    B.LinkDirectionalArrowRelPos, "link directional arrow relative position", "value"
)

if __name__ == "__main__":
    add_link_directional_arrow_rel_pos(fg, box)
    display(box)

### `LinkDirectionalParticleColor`

In [ ]:
add_link_directional_particle_color = make_link_behavior_with_ui(
    B.LinkDirectionalParticleColor,
    "link directional particle color",
    "color",
    is_color=True,
)

if __name__ == "__main__":
    add_link_directional_particle_color(fg, box)
    display(box)

### `LinkDirectionalParticleSpeed`

In [ ]:
add_link_directional_particle_speed = make_link_behavior_with_ui(
    B.LinkDirectionalParticleSpeed, "link directional particle speed", "value"
)

if __name__ == "__main__":
    add_link_directional_particle_speed(fg, box)
    display(box)

### `LinkDirectionalParticleWidth`

In [ ]:
add_link_directional_particle_width = make_link_behavior_with_ui(
    B.LinkDirectionalParticleWidth, "link directional particle width", "value"
)

if __name__ == "__main__":
    add_link_directional_particle_width(fg, box)
    display(box)

### `LinkDirectionalParticles`

In [ ]:
add_link_directional_particles = make_link_behavior_with_ui(
    B.LinkDirectionalParticles, "link directional particles", "value"
)

if __name__ == "__main__":
    add_link_directional_particles(fg, box)
    display(box)